# TopoStats - Minicircle Walk-Through

Welcome, this [Jupyter Notebook](https://jupyter.org/) will take you through processing `minicricle.spm`

## Installing TopoStats

There are several different ways of installing TopoStats depending on what you want to do. The simplest is to install
from GitHub under a virtual environment.

```bash
pip install git+https://github.com/AFM-SPM/TopoStats.git@<branch>
```

For more information on the different ways of installing TopoStats and setting up Virtual Environments please refer to
[installation](https://afm-spm.github.io/TopoStats/installation.html).



### Running in Colab

If you are running this Notebook in [Google Colab](https://colab.research.google.com/) then you will have to execute the
following cell in order to install TopoStats.

In [ ]:
pip install git+https://github.com/AFM-SPM/TopoStats.git@ns-rse/242a-notebooks

## Getting Started


### Loading Libraries and Modules

TopoStats is written as a series of modules with various classes and functions. In order to use these interactively we
need to `import` them.

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from topostats.filters import Filters
from topostats.grains import Grains
from topostats.grainstats import GrainStats
from topostats.io import find_files, read_yaml, write_yaml, LoadScans
from topostats.logs.logs import setup_logger, LOGGER_NAME
from topostats.plottingfuncs import Images
from topostats.tracing.dnatracing import dnaTrace, traceStats
from topostats.utils import update_config

## Finding Files

TopoStats needs to find files to process and the `find_images()` function helps here. It takes as a argument the
directory path that should be searched and the file extension to look for (currently only `.spm` is supported) and
returns a list of all files that are found that have that file extension in the specified directory.

In [ ]:
# Set the base directory to be current working directory of the Notebook
BASE_DIR = Path().cwd()
FILE_EXT = ".spm"
# Search for *.spm files in this directory
spm_files = find_files(base_dir=BASE_DIR.parent / "tests", file_ext=FILE_EXT)

`spm_files` is a list of images that match and we can look at that list.

In [ ]:
spm_files

If you wish to search for files in a different directory such as `/home/user/work/my_scans` then you should adapt the
above.

```python
BASE_DIR = Path("/home/user/work/my_scans")
FILE_EXT = ".spm"
spm_files = find_images(base_dir=BASE_DIR, file_ext=FILE_EXT)
```


## Loading a Configuration

You can specify all options explicitly by hand when instantiating classes or calling methods/functions. However when run
at the command line in batch mode TopoStats loads these options from a [YAML](https://yaml.org/) configuration file and it is worth
understanding the structure of this file and how it is used.

A trimmed version is shown below. The words that come before the colon `:` are the option, e.g. `base_dir:` is the base
directory that is searched for files, what comes after is the value, in this case `./tests/`


```yaml
base_dir: ./
output_dir: ./output
warnings: ignore
cores: 2
quiet: false
file_ext: .spm
loading:
  channel: Height
filter:
  run: true # Options : true, false
  row_alignment_quantile: 0.5 # Lower values may improve flattening of larger features
  threshold_method: std_dev # Options : otsu, std_dev, absolute
  otsu_threshold_multiplier: 1.0
  threshold_std_dev:
    lower: 10.0
    upper: 1.0
  threshold_absolute:
    lower: -1.0
    upper: 1.0
  gaussian_size: 1.0121397464510862 # in px
  gaussian_mode: nearest
  remove_scars:
    run: true
    removal_iterations: 2
    threshold_low: 0.250
    threshold_high: 0.666
    max_scar_width: 4
    min_scar_length: 16
grains:
  run: true # Options : true, false
  # Thresholding by height
  threshold_method: std_dev # Options : std_dev, otsu, absolute
  otsu_threshold_multiplier: 1.0
  threshold_std_dev:
    lower: 10.0
    upper: 1.0
  threshold_absolute:
    lower: -1.0
    upper: 1.0
  direction: upper # Options: upper, lower, both (defines whether to look for grains above or below thresholds or both)
  # Thresholding by area
  smallest_grain_size_nm2: 50 # Size in nm^2 of tiny grains/blobs (noise) to remove, must be > 0.0
  absolute_area_threshold:
    upper: [ 300, 3000 ] # above surface [Low, High] in nm^2 (also takes null)
    lower: [ null, null ] # below surface [Low, High] in nm^2 (also takes null)
grainstats:
  run: true # Options : true, false
  edge_detection_method: binary_erosion # Options: canny, binary erosion. Do not change this unless you are sure of what this will do.
  cropped_size: 40.0 # Length (in nm) of square cropped images (can take -1 for grain-sized box)
dnatracing:
  run: true # Options : true, false
plotting:
  run: true # Options : true, false
  save_format: png # Options : see https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.savefig.html
  image_set: core  # Options : all, core
  zrange: [null, null]  # low and high height range for core images (can take [null, null])
  colorbar: true  # Options : true, false
  axes: true # Options : true, false (due to off being a bool when parsed)
  cmap: nanoscope # Options : nanoscope, afmhot, gwyddion
  mask_cmap: blu # Options : blu, jet_r and any in matplotlib
  histogram_log_axis: false # Options : true, false
  histogram_bins: 200 # Number of bins for histogram plots to use
summary_stats:
  run: true
  config: null
```

To load the configuration file into Python we use the `read_yaml()` function. This saves the options as a dictionary and
we can access values by the keys. The example below prints out the top-levels keys and then the keys for the `filter`
configuration. 

**NB** Python dictionaries have keys which can be considered as the parameter that is to be configured and each key has
an associated value which is the value you wish to set the parameter to.

In [ ]:
from topostats.io import read_yaml
config = read_yaml(BASE_DIR.parent / "topostats" / "default_config.yaml")
print(f"Top level keys of config.ymal : \n\n {config.keys()}\n")
print(f"Configuration options for Filter : \n\n {config['filter'].keys()}")

You can look at all of the options using the `json` package to "pretty" print the dictionary which makes it easier to
read. Here we print the `filter` section. You can see the options map to those of the `Filter()` class with an
additional `"run": true` which is used when running TopoStats at the command line.

In [ ]:
print(json.dumps(config["filter"], indent=4))

We will use the configuration options we have loaded in processing the `minicircle.spm` image. For convenience we save
each set of options to their own dictionary and remove the `run` entry as this is not required when running TopoStats
interactively.


In [ ]:
loading_config = config["loading"]
filter_config = config["filter"]
filter_config.pop("run")
grain_config = config["grains"]
grain_config.pop("run")
grainstats_config = config["grainstats"]
grainstats_config.pop("run")
dnatracing_config = config["dnatracing"]
dnatracing_config.pop("run")
plotting_config = config["plotting"]
plotting_config.pop("run")

## Load Scans

The first step before processing is to load a scan, this extracts the image data to a Numpy array along with the
filepath and the pixel to nanometer scaling parameter which is used to correctly scale the pixels to images. These are
stored in nested dictionaries with one top-level entry for each image that is found.

In [ ]:
all_scan_data = LoadScans(spm_files, **config["loading"])
all_scan_data.get_data()
all_scan_data.img_dict

Now that we have loaded the data we can start to process it. The first step is filtering the image.

## Filter Image

Now that we have found some images the first step in processing is to filter them to remove some of the noise. This is
achieved using the `Filters()` class.  There are number of options that we need to specify which are described in the
table below and also in the [documentation](https://topostats.readthedocs.io/en/dev/topostats.filters.html). 

If you look carefully you will see there is mostly a one-to-one mapping between the options and the keys in the
configuration dictionary. 

| Option                      | Type               | Description                                   | Default |
|:----------------------------|:-------------------|:----------------------------------------------|:--------|
| `img_path`                  | `Union[str, Path]` | The path to the image that is to be filtered. |         |
| `threshold_method`          | `str`              |                                               |         |
| `otsu_threshold_multiplier` | `float`            |                                               | 1.7     |
| `threshold_std_dev`         | `float`            |                                               | None    |
| `threshold_absolute_lower`  | `float`            |                                               | None    |
| `threshold_absolute_upper`  | `float`            |                                               | None    |
| `channel`                   | `str`              |                                               |         |
| `amplify_level`             | `float`            |                                               | `None`  |
| `output_dir`                | `Union[str, Path]` |                                               | `None`  |
| `quiet`                     | `bool`             |                                               | `False` |

One of the key fields you may wish to change is the `channel`.

Once we setup a `Filters` object we can call the different methods that are available for it. There are lots of
different methods that carry out the different steps but for convenience the `filter_image()` method runs all these.

The following section instantiates ("sets up") an object called `filtered_image` of type `Filters` using the first file
found (`spm_files[0]`) and the various options from the dictionary `filter_config` dictionary.


In [ ]:
from topostats.filters import Filters
filtered_image = Filters(image=all_scan_data.img_dic["minicircle"]["image"],
                         filename=all_scan_data.img_dic["minicircle"]["img_path"],
                         pixel_to_nm_scaling=all_scan_data.img_dic["minicircle"]["px_2_nm"],
                         row_alignment_quantile=filter_config["row_alignment_quantile"],
                         threshold_method=filter_config["threshold_method"],
                         otsu_threshold_multiplier=filter_config["otsu_threshold_multiplier"],
                         threshold_std_dev=filter_config["threshold_std_dev"],
                         threshold_absolute=filter_config["threshold_absolute"],
                         gaussian_size=filter_config["gaussian_size"],
                         gaussian_mode=filter_config["gaussian_mode"],
                         remove_scars=filter_config["remove_scars"])


# NB - Because of the one-to-one mapping of configuration options to Filters() options we can use keyword arguments to
#      unpack the options, the below is the same as explicitly stating the values you want to map.
filtered_image = Filters(image=all_scan_data.img_dic["minicircle"]["image"],
                         filename=all_scan_data.img_dic["minicircle"]["img_path"],
                         pixel_to_nm_scaling=all_scan_data.img_dic["minicircle"]["px_2_nm"],
                         **filter_config)
filtered_image.filter_image()


The `filtered_image` now has a a number of NumPy arrays saved in the `.images` dictionary that can be accessed and plotted. To view
the names of the images (technically the dictionary keys) you can print them with `filter_image.images.keys()`...

In [ ]:
print(f"Available NumPy arrays to plot in filter_image.images dictionary :\n\n{filtered_image.images.keys()}")

To plot the raw extracted pixels you can use the built-in NumPy method `imshow()`.


In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
plt.imshow(filtered_image.images["pixels"], cmap='hot')
plt.show()

TopoStats includes a custom plotting class `Images`  which formats plots in a more familiar manner.

It has a number of options, please refer to the official documentation on
[configuration](https://afm-spm.github.io/TopoStats/configuration.html) under the `plotting` entry for what these values
are or the [API
reference](https://afm-spm.github.io/TopoStats/topostats.plottingfuncs.html#module-topostats.plottingfuncs).

The class requires a Numpy array, which we have just generated many of during the various filtering stages, and a number
of options. Again for convenience we use the `**plotting_config` notation to unpack the key/value pairs stored in the
`plotting_config` dictionary.




In [ ]:
## TODO - Not getting fig, ax returned by plot_and_save()?
fig, ax = Images(data=filtered_image.images["pixels"],
                        filename=filtered_image.filename,
                        output_dir="img/",                   
                        save=True,
                        **plotting_config).plot_and_save()
fig

Here we plot the image after processing and zero-averaging the background but with the `viridis` palette and
constraining the `zrange` to be between 0 and 3

In [ ]:
## TODO - Not getting fig, ax returned by plot_and_save()?
# First remove the current value for cmap in the plotting_config dictionary, otherwise an error occurs because the same
# argument will have been specified twice.
current_cmap = plotting_config.pop("cmap")
fig, ax = Images(data=filtered_image.images["pixels"],
                      filename=filtered_image.filename,
                      output_dir="img/",                   
                      save=True,
                      **plotting_config).plot_and_save()
# Restore the value for cmap to the dictionary.
plotting_config["cmap"] = current_cmap
fig

## Finding Grains

The next step in processing the image is to find grains. This is done using the `Grains` class and we have saved the
configuration to the `grains_config` dictionary. For details of the arguments and their values please refer to the
[configuration](https://afm-spm.github.io/TopoStats/configuration.html) and the [API
reference](https://afm-spm.github.io/TopoStats/topostats.grains.html).

The most important thing required for grain finding is the resulting image from the Filtering stage, however several
other key variables are required. Again there is a one-to-one mapping between the options to the `Grains()` class and
their values in the configuration file.

The `pixel_to_nm_scaling` is one of several key variables, as is the `filename` and whilst you can specify these
yourself explicitly they are available as properties of the `filtered_image` that we have processed. As with `Filters`
the `Grains` class has a number of methods that carry out the grain finding, but there is a convenience method
`find_grains()` which calls all these in the correct order.

In [ ]:
from topostats.grains import Grains
grains = Grains(image=filtered_image.images["zero_average_background"],
                filename=filtered_image.filename,
                pixel_to_nm_scaling=filtered_image.pixel_to_nm_scaling,
                threshold_method=grain_config["threshold_method"],
                otsu_threshold_multiplier=grain_config["otsu_threshold_multiplier"],
                threshold_std_dev=grain_config["threshold_std_dev"],
                threshold_absolute=grain_config["threshold_absolute"],
                direction=grain_config["direction"],
                smallest_grain_size_nm2=grain_config["smallest_grain_size_nm2"],
                )
# NB - Again we can use the one-to-one mapping of configuration options in the grain_config we extracted.
grains = Grains(image=filtered_image.images["zero_average_background"],
                filename=filtered_image.filename,
                pixel_to_nm_scaling=filtered_image.pixel_to_nm_scaling,
                **grain_config)
grains.find_grains()


The `grains` object now also contains a series of images in the `grains.images` dictionary that we can plot.

In [ ]:
print(f"Available NumPy arrays to plot in grains.images dictionary :\n\n{grains.images.keys()}")

And we can again use the `plot_and_save()` function to plot these.

In [ ]:
## TODO - Not getting fig, ax returned by plot_and_save()?
fig, ax = Images(data=grains.images["objects_removed"],
                      filename=filtered_image.filename,
                      output_dir="img/",                   
                      save=True,
                      **plotting_config).plot_and_save()
fig

### Thresholds

The thresholds can be used in different ways based on the `direction` you want to detect grains. Typically for molecular
imaging where the DNA or protein is raised above the background you will want to look for items above the `upper`
threshold. When imaging silicon though you may be interested in items below a `lower` threshold. For convenience it is
possible to look for grains that are `both` above the `upper` threshold and `lower` than the lower threshold.

This is controlled using the `config["grains"]["direction"]` configuration option which maps to the `Grains(direction=)`
option and can take three values `lower`, `upper` or `both`.

If you want to change the option you can update the `config["grains"]` dictionary as we do below. You will see that now
we are using `both` there is twice as much output as grains are detected above the reported upper Threshold (0.8064)
_and_ below the reported lower Threshold (-0.5333).

In [ ]:
grain_config["direction"] = "both"
grains_both = Grains(image=filtered_image.images["zero_average_background"],
                filename=filtered_image.filename,
                pixel_to_nm_scaling=filtered_image.pixel_to_nm_scaling,
                **grain_config)
grains.find_grains()


There above method is using `threshold_method="std_dev"` defined in the configuration file we loaded. This calculates the
mean and standard deviation of height across the whole image and then determines the threshold by scaling the standard
deviation by a given factor (defined by `threshold_std_dev`) and adding it to the mean to give the `upper` threshold
and/or subtracting if from the mean to give the `lower` threshold.

An alternative method is to use the `threshold_method="absolute"`, set the `direction=upper` and explicitly state the
`lower` and `upper` thresholds (although since we are only looking for objects above a given threshold only the `upper`
value will be used). If you wish to specify values for both they are shown below.

In [ ]:
## ToDO - What are sensible values for minicircle here?
grain_config["threshold_method"] = "absolute"
grain_config["direction"] = "upper"
grain_config["threshold_absolute"]["upper"] = 0.01   # Change just the upper threshold
grain_config["threshold_absolute"]["lower"] = -2.0  # Change just the lower threshold
grain_config["threshold_absolute"]["upper"] = {"lower": -2.0, "upper": 1.2}  # Change just the both lower and upper threshold
grains_absolute = Grains(image=filtered_image.images["zero_average_background"],
                filename=filtered_image.filename,
                pixel_to_nm_scaling=filtered_image.pixel_to_nm_scaling,
                **grain_config)
grains_absolute.find_grains()

This is important to know because you need to know where the resulting images are stored within the `Grains.direction`
dictionary. This will have entries corresponding to the `direction` that grains have been searched for.

In [ ]:
print(f"Grains available in original 'grains' (std_dev, upper) : {grains.directions.keys()}")
print(f"Grains available in both (std_dev, both)               : {grains_both.directions.keys()}")
print(f"Grains available in absolute (absolute, upper)         : {grains_absolute.directions.keys()}")

Each `direction` dictionary is a series of NumPy arrays representing the cleaned images and these can be plotted.

In [ ]:
print(f"Images available for plotting : {grains.directions['upper'].keys()}")
fig, ax = Images(data=grains.directions["upper"]["mask_grains"],
                 filename=filtered_image.filename,
                 output_dir="img/",                   
                 save=True,
                 **plotting_config).plot_and_save()
fig

In [ ]:
fig, ax = Images(data=grains.directions["upper"]["removed_small_objects"],
                 filename=filtered_image.filename,
                 output_dir="img/",                   
                 save=True,
                 **plotting_config).plot_and_save()
fig

In [ ]:
fig, ax = Images(data=grains.directions["upper"]["coloured_regions"],
                 filename=filtered_image.filename,
                 output_dir="img/",                   
                 save=True,
                 **plotting_config).plot_and_save()
fig

## Grain Statistics

Now that grains have been found we can calculate statistics on each of the grains. This is done using the `GrainStats()`
class. Again the configuration options from the YAML file map to those of the class and there is a convenience method
`calculate_stats()` which will run all steps of grain finding. However, because the class is processing results that we
have generated we have to explicitly pass in more values.

For details of what the arguments are please refer to the [API
reference](https://afm-spm.github.io/TopoStats/topostats.grainstats.html).

The `GrainStats` class returns two objects, a Pandas `pd.DataFrame` of calculated statistics and a `list` of
dictionaries containing the grain data to be plotted. We therefore instantiate ("set-up") the `grain_stats` dictionary
to hold these results and unpack each to the keys `statistics` and `plots` respectively.



In [ ]:
grain_stats 
grainstats = GrainStats(data=filtered_image.images["gaussian_filtered"],
                        labelled_data=grains.directions["upper"]["labelled_regions_02"],
                        pixel_to_nanometre_scaling=filtered_image.pixel_to_nm_scaling,
                        direction="upper",
                        base_output_dir="grains",
                        image_name=filtered_image.filename,
                        **grainstats_config)
# grain_stats["statistics"], grain_stats["plots"] = grainstats.calculate_stats()
_temp = grainstats.calculate_stats()
grain_stats = {"statistics": _temp[0], "plots": _temp[1]}

The `statistics` is a [Pandas
DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). We can print this out as
shown below.

In [ ]:
grain_stats["statistics"]

Further we can summarise the dataframe or a subset of varaibles.

In [ ]:
grain_stats["statistics"][["smallest_bounding_width", "aspect_ratio"]].describe()

In [ ]:
### Plotting Individual Grains

It is possible to plot the individual grains in the same way that whole images are plotted. These are now stored as the
`grain_stats["plots"]` dictionary. We can find out how many grains there are either by looking at the number of rows
reported when the statisics Pandas Data Frame was printed above or its summary (from `.describe()`), which reports the
`count`.

We can then plot any of these using the number indexing of the list. We using
[f-strings](https://realpython.com/python-f-strings/) as the value for the argument `filename` to fill in the images
filename automatically and append the `_grain#` for the image numbe we are plotting, in this example `0`. Try plotting
#some other images.

In [ ]:
fig, ax = Images(data=grain_stats["plots"][0],
                 filename=f"{filtered_image.filename}_grain0",
                 output_dir="img/",                   
                 save=True,
                 **plotting_config).plot_and_save()
fig

## DNA Tracing

When working with molecules it is possible to calculate DNA Tracing Statistics using the `dnaTrace` and `traceStats`
classes.

**NB** These classes are likely to undergo refactoring and change in the coming months, this Notebook will be updated
in light of changes. Currently there are very few options that map to the configuration. For that reason the options are
not currently detailed and described in this Notebook, although the [API
Documentation](https://afm-spm.github.io/TopoStats/topostats.tracing.html) details them.

Currently this is a two stage process

1. `dnaTrace` traces molecules to produce a "trace_object".
2. `traceStats` takes the "trace_object" and produces a Pandas DataFrame

In [ ]:
dna_traces = dnaTrace(full_image_data=filtered_image.images["gaussian_filtered"],
                      grains=grains.directions["upper"]["labelled_regions_02"],
                      filename=grains.filename,
                      pixel_size=grains.pixel_to_nm_scaling,
                      **dnatracing_config)
dna_traces.trace_dna()
tracing_stats = traceStats(trace_object=dna_traces,
                           image_path=spm_files[0])

The resulting Pandas DataFrame is stored in the `df` attribute of the `traceStats` object, to view it we...

In [ ]:
tracing_stats.df

### Merge GrainStats and TracingStats

Its reasonable to want to have a single data set with which we work and so we now merge the GrainStats data frame with
the Tracing Statistics and then save them to CSV for subsequent use. The following saves them to the directory in which
the Notebook is running with the filename `minicircle_example.csv`

In [ ]:
all_stats_df = grain_stats["statistics"].merge(tracing_stats.df, on=["image", "molecule_number"])
all_stats_df.to_csv("minicircle_example.csv")

These statistics now be plotted to show the distribution of the different metrics. Please see the Jupyter Notebook
`notebooks/01-Summary-statistics-and-plots.ipynb` for examples of how to plot these statistics.